# IAM UBA Workshop 2019

## 7. Compute the Social Cost of Carbon

In this final section we will compute the Social Cost of Carbon with all the models that we have used in this workshop.

### Base calculations

First, let's load all the models we want to use for this exercise:

In [1]:
using Mimi, MimiDICE2010, MimiDICE2013, MimiRICE2010, MimiPAGE2009, MimiFUND

We can then call the `compute_scc` function for each model to get a SCC estimate for that model. Note that we have to pass the year of the marginal emission pulse as a keyword argument to that function call. Note that the `compute_scc` function is a function that model creators have added individually to each of these models in an effort to maintain a consistent and useful API of commonly used functions, but is not inherent to Mimi itself.

In [2]:
MimiDICE2010.compute_scc(year=2015)

16.39815430060173

In [3]:
MimiDICE2013.compute_scc(year=2015)

20.8723234611904

In [4]:
MimiPAGE2009.compute_scc(year=2010)

52.326389484803975

In [5]:
MimiRICE2010.compute_scc(year=2015)

14.546230704004822

In [6]:
MimiFUND.compute_scco2(year=2015)

1.5928348049789185

### Standardize across models

We are still comparing apples and oranges, though, because we haven't adjusted price levels, nor discounting schemes. Lets try to come up with estimates that are more comparable by using a constant 3% discount rate for all models. We can achieve that by setting the elasticity of marginal utility with respect to consumption to 0 (e.g. `eta=0.`) and then using a 3% pure rate of time preference.

Finally, we will also convert all price levels to 2019.

In [7]:
MimiDICE2010.compute_scc(year=2015, prtp=0.03, eta=0.) * 1.34 # Convert from 2005 to 2019 $

49.877542928529714

In [8]:
MimiDICE2013.compute_scc(year=2015, prtp=0.03, eta=0.) * 1.34 # Convert from 2005 to 2019 $

62.679925156351

In [9]:
MimiRICE2010.compute_scc(year=2015, prtp=0.03, eta=0.) * 1.34 # Convert from 2005 to 2019 $

42.5465928983676

In [10]:
MimiPAGE2009.compute_scc(year=2010, prtp=0.03, eta=0.) * 1.21 # Convert from 2008 to 2019 $

39.85041042211828

In [11]:
MimiFUND.compute_sc(year=2015, gas=:CO2, prtp=0.03, eta=0.) * 1.7 # Convert from 1995 to 2019 $

9.930283935320011

### With a modified model

The precanned `compute_scc` functions work well if we want to run the models as they were built by their creators, but sometimes we want to compute the SCC for a modified model. In the following example we will compute the SCC with FUND for a climate sensitivity of 6.

To do so, we first create a new FUND model instance:

In [12]:
m_fund = MimiFUND.get_model()

29-component Mimi.Model:
  scenariouncertainty::MimiFUND.scenariouncertainty
  population::MimiFUND.population
  geography::MimiFUND.geography
  socioeconomic::MimiFUND.socioeconomic
  emissions::MimiFUND.emissions
  climateco2cycle::MimiFUND.climateco2cycle
  climatech4cycle::MimiFUND.climatech4cycle
  climaten2ocycle::MimiFUND.climaten2ocycle
  climatesf6cycle::MimiFUND.climatesf6cycle
  climateforcing::MimiFUND.climateforcing
  climatedynamics::MimiFUND.climatedynamics
  biodiversity::MimiFUND.biodiversity
  climateregional::MimiFUND.climateregional
  ocean::MimiFUND.ocean
  impactagriculture::MimiFUND.impactagriculture
  impactbiodiversity::MimiFUND.impactbiodiversity
  impactcardiovascularrespiratory::MimiFUND.impactcardiovascularrespiratory
  impactcooling::MimiFUND.impactcooling
  impactdiarrhoea::MimiFUND.impactdiarrhoea
  impactextratropicalstorms::MimiFUND.impactextratropicalstorms
  impactforests::MimiFUND.impactforests
  impactheating::MimiFUND.impactheating
  impactvectorb

Next, we modify the climate sensitivity parameter in this model using the `set_param!` function that we used previously:

In [13]:
set_param!(m_fund, :climatedynamics, :climatesensitivity, 6.)

And now we can call the `compute_scc` function again, but pass our modified model as the first argument. In that case, `compute_scc` will use the modificatoins we made to the model when it estimates the SCC:

In [16]:
MimiFUND.compute_sc(m_fund, year=2015, gas=:CO2, eta=0., prtp=0.03) * 1.7

15.483425727840777

### With equity weighting

The equity weighting options currently only works for FUND. You can compute an equity weighted SCC with the following code:

In [15]:
MimiFUND.compute_sc(year=2015, gas=:CO2, eta=0., prtp=0.03, equity_weights=true, equity_weights_normalization_region=3)

5.841343491364714

The `equity_weights_normalization_region` parameter controls how the equity weighted SCC is normalized. A value of `0` amounts to using a world average per capita value for the normalization. Any other value refers to an index of a region whos per capita values are used for the normalization.